In [1]:
from ann import read_bank_note, get_error, get_sign

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
class FC_He_Relu(nn.Module):

    def __init__(self, din, dout):
        super().__init__()

        self.fc = nn.Linear(din, dout)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):

        x = self.fc(x)
        x = torch.relu(x)

        return x

In [5]:
class FC_Xavier_Tanh(nn.Module):

    def __init__(self, din, dout):
        super().__init__()

        self.fc = nn.Linear(din, dout)
        nn.init.xavier_normal_(self.fc.weight)

    def forward(self, x):

        x = self.fc(x)
        x = torch.tanh(x)

        return x

In [6]:
class GetNetwork(nn.Module):
    def __init__(self, din, width=5, dout=1, depth=3, layer=FC_Xavier_Tanh):
        super().__init__()

        first_layer = [layer(din, width)]

        depth -=2
        middle_layers = [layer(width, width) for i in range(depth)]

        last_layer = [layer(width, dout)]

        layers = first_layer + middle_layers + last_layer
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [7]:
def train(model, optimizer, X, y, epochs, device='cpu'):
    
    model = model.to(device=device)

    N, D = X.shape
    
    X = X.to(device=device)
    y = y.to(device=device)
#     y = y.reshape(-1, 1)

    training_loss=[]
    for e in range(epochs):

        for b in range(N):
            xi = X[b].unsqueeze(dim=0)
#             yi = y[b]
            yi = y[b].reshape(1, -1)
            
#             xi = xi.to(device=device)
#             yi = yi.to(device=device)

            scores = model(xi)

            F.mse_loss(scores, yi)

            loss = F.mse_loss(scores, yi)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

        with torch.no_grad():
            scores = model(X)
            training_loss.append(F.mse_loss(scores, y.reshape(-1, 1)).item())

    return training_loss

In [8]:
def get_error_nn(model, X, y):
    
    with torch.no_grad():
        y_pred = get_sign(model(X))

    y_true = np.array(y)

    return get_error(y_pred, y_true)

In [9]:
# if torch.cuda.is_available():
#     device = torch.device('cuda:0')
# else:
#     device = torch.device('cpu')
    
# print(f'Using: {device}')

In [10]:
X_train, y_train = read_bank_note(test=False)
X_test, y_test = read_bank_note(test=True)

dtype = torch.float32

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

X_train = X_train.to(dtype=dtype)
X_test = X_test.to(dtype=dtype)
y_train = y_train.to(dtype=dtype)
y_test = y_test.to(dtype=dtype)

In [11]:
N, D = X_train.shape

In [12]:
epochs=100

depth_values = [3, 5, 9]
width_values = [5, 10, 25, 50, 100]

rec_depth=[]
rec_width=[]
rec_train=[]
rec_test=[]

for depth in depth_values:
    for width in width_values:
        
        print(f"Starting Xavier depth: {depth} width: {width}")

        model = GetNetwork(din=D, width=width, dout=1, depth=depth, layer=FC_Xavier_Tanh)
        optimizer = optim.Adam(model.parameters())

        loss = train(model, optimizer, X_train, y_train, epochs=epochs)
        
        rec_depth.append(depth)
        rec_width.append(width)
        rec_train.append(get_error_nn(model, X_train, y_train))
        rec_test.append(get_error_nn(model, X_test, y_test))
        
print('Done\n')

Starting Xavier depth: 3 width: 5
Starting Xavier depth: 3 width: 10
Starting Xavier depth: 3 width: 25
Starting Xavier depth: 3 width: 50
Starting Xavier depth: 3 width: 100
Starting Xavier depth: 5 width: 5
Starting Xavier depth: 5 width: 10
Starting Xavier depth: 5 width: 25
Starting Xavier depth: 5 width: 50
Starting Xavier depth: 5 width: 100
Starting Xavier depth: 9 width: 5
Starting Xavier depth: 9 width: 10
Starting Xavier depth: 9 width: 25
Starting Xavier depth: 9 width: 50
Starting Xavier depth: 9 width: 100
Done



In [13]:
df_results = pd.DataFrame({'Depth': rec_depth, 'Width': rec_width, 'TrainError': rec_train, 'TestError': rec_test})
df_results.round(3).to_csv('results_pytorch_xavier_100.csv', index=False)
df_results.round(3)

,Depth,Width,TrainError,TestError
0,3,5,0.000,0.000
1,3,10,0.000,0.000
2,3,25,0.000,0.000
3,3,50,0.000,0.000
4,3,100,0.000,0.000
5,5,5,0.000,0.000
6,5,10,0.000,0.002
7,5,25,0.000,0.000
8,5,50,0.000,0.000
9,5,100,0.000,0.000


In [14]:
epochs=100

depth_values = [3, 5, 9]
width_values = [5, 10, 25, 50, 100]

rec_depth=[]
rec_width=[]
rec_train=[]
rec_test=[]

for depth in depth_values:
    for width in width_values:
        
        print(f"Starting He depth: {depth} width: {width}")

        model = GetNetwork(din=D, width=width, dout=1, depth=depth, layer=FC_He_Relu)
        optimizer = optim.Adam(model.parameters())

        loss = train(model, optimizer, X_train, y_train, epochs=epochs)
        
        rec_depth.append(depth)
        rec_width.append(width)
        rec_train.append(get_error_nn(model, X_train, y_train))
        rec_test.append(get_error_nn(model, X_test, y_test))
print('Done\n')

Starting He depth: 3 width: 5
Starting He depth: 3 width: 10
Starting He depth: 3 width: 25
Starting He depth: 3 width: 50
Starting He depth: 3 width: 100
Starting He depth: 5 width: 5
Starting He depth: 5 width: 10
Starting He depth: 5 width: 25
Starting He depth: 5 width: 50
Starting He depth: 5 width: 100
Starting He depth: 9 width: 5
Starting He depth: 9 width: 10
Starting He depth: 9 width: 25
Starting He depth: 9 width: 50
Starting He depth: 9 width: 100
Done



In [15]:
df_results = pd.DataFrame({'Depth': rec_depth, 'Width': rec_width, 'TrainError': rec_train, 'TestError': rec_test})
df_results.round(3).to_csv('results_pytorch_he_100.csv', index=False)
df_results.round(3)

,Depth,Width,TrainError,TestError
0,3,5,0.000,0.000
1,3,10,0.000,0.000
2,3,25,0.446,0.442
3,3,50,0.446,0.442
4,3,100,0.000,0.000
5,5,5,0.446,0.442
6,5,10,0.000,0.000
7,5,25,0.446,0.442
8,5,50,0.446,0.442
9,5,100,0.446,0.442
